In [1]:
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/My \Drive/Sem \8 \files/
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
/content/drive/My Drive/Sem 8 files
 data   euclidean_distances.json  'Fri Feb 14 09_06_16 2020.json'


In [2]:
!apt-get install -y libspatialindex-c4v5
!pip3.6 install osmnx

import osmnx as ox
import matplotlib.pyplot as plt
import networkx as nx
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import math

from shapely.geometry import box
from heapq import heappush, heappop
from itertools import count
from rtree import index

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libspatialindex4v5
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex4v5
0 upgraded, 2 newly installed, 0 to remove and 25 not upgraded.
Need to get 270 kB of archives.
After this operation, 1,107 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex4v5 amd64 1.8.5-5 [219 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-c4v5 amd64 1.8.5-5 [51.7 kB]
Fetched 270 kB in 0s (3,091 kB/s)
Selecting previously unselected package libspatialindex4v5:amd64.
(Reading database ... 145118 files and directories currently installed.)
Preparing to unpack .../libspatialindex4v5_1.8.5-5_amd64.deb ...
Unpacking libspatialindex4v5:amd64 (1.8.5-5) ...
Selecting previously unselected package libspatialindex-c4v5:amd64.
Preparing to

In [3]:
place_name = "Singapore, Central, Singapore"
G = ox.save_load.load_graphml(filename="Singapore_drive_processed.graphml")

# Stores nodes and edges mapped with speed bands along with other attributes
type(G)

networkx.classes.multidigraph.MultiDiGraph

In [0]:
edges = ox.graph_to_gdfs(G, nodes=False, edges=True)
nodes = pd.read_pickle('data/nodes_drive.pkl')

In [0]:
def process_speed_band(df):
    
    # Define Locations
    def truncate(number, digits) -> float:
        stepper = 10.0 ** digits
        return math.trunc(stepper * number) / stepper
    
    location = df['Location'].values
    i = 0
    
    for loc in location:
        x1, y1, x2, y2 = [float(n) for n in loc.split(' ')]
        
        x1 = truncate(x1, 7)
        y1 = truncate(y1, 7)
        x2 = truncate(x2, 7)
        y2 = truncate(y2, 7)
        
        if y1 < y2:
            bottom = y1
            top = y2
        else:
            bottom = y2
            top = y1
            
        if x1 < x2:
            left = x1
            right = x2
        else:
            left = x2
            right = x1
        
        df['Location'].values[i] = (left, bottom, right, top)
        i += 1 
    
    # Process maximum speeds
    i = 0
    for x in df['SpeedBand']:
        if x == 0:
            df['MaximumSpeed'][i] = '50'
        i += 1 

In [6]:
f = open("Fri Feb 14 09_06_16 2020.json", "r").read()
x = json.loads(f)

speed_bands = pd.DataFrame.from_dict(x, orient='columns')
process_speed_band(speed_bands)
# speed_bands.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
def calculate_congestion(G, edges, speed_bands):
    # Define Location    
    location = edges['geometry'].values
    i = 0
    Location = []
    
    for loc in location:        
        x1 = loc.xy[1][-1]
        y1 = loc.xy[0][-1]
        x2 = loc.xy[1][0]
        y2 = loc.xy[0][0]
        
        if y1 < y2:
            bottom = y1
            top = y2
        else:
            bottom = y2
            top = y1
            
        if x1 < x2:
            left = x1
            right = x2
        else:
            left = x2
            right = x1
        
        Location.append((left, bottom, right, top))
        i += 1 
        
    edges['Location'] = Location
    
    
    # Define observed_speed using speedband dataset 
    
    # key: maximum speed ,value: location
    idx = index.Index()
    Location = speed_bands['Location']
    Speed = speed_bands['MaximumSpeed'].astype(float)

    for speed, loc in zip(Speed, Location):
        idx.insert(int(speed), loc)
    
    # Find intersections
    observed_speed = []
    Location = edges['Location']
    speeds =  edges['maxspeed']

    for loc, speed in zip(Location, speeds):
      max_speeds = list(idx.intersection(loc))

      if len(max_speeds) == 0: # edge didn't intersect with any speed band
          observed_speed.append(speed)
      else:
          observed_speed.append(sum(max_speeds)/len(max_speeds))

    edges['observed_speed'] = observed_speed
    
    
    # Define BPR heuristic for each road link
    bpr = dict()
    n = len(edges['u'])

    for i in range (n):
        u = edges['u'][i]
        v = edges['v'][i]
        key = edges['key'][i]
        time = float(edges['travel_time'][i])
        flow = float(edges['observed_speed'][i])
        capacity = float(edges['maxspeed'][i])

        bpr[(u,v,key)] = time * (1 + 0.15*(flow/capacity)**4)
        
    nx.set_edge_attributes(G, bpr, 'BPR')


    # Define capacity as a random number between 835-905
    import random
    capacity  = dict()

    for i in range(n):
        x = random.randint(835, 905)
        u = edges['u'][i]
        v = edges['v'][i]
        key = edges['key'][i]
        capacity[(u,v,key)] = x

    nx.set_edge_attributes(G, capacity, 'capacity')

In [0]:
calculate_congestion(G, edges, speed_bands)
# edges.head()

### Finding shortest route between centroid of map and another point

In [9]:
# Boundary Box of entire area
bbox = box(*edges.unary_union.bounds)
orig_point = bbox.centroid
print(orig_point)

POINT (103.82129505 1.3515309)


In [10]:
target_point = nodes[nodes['osmid'] == 1820000257].geometry.values[0]
print(target_point)

POINT (103.9326163 1.3565805)


In [0]:
orig_xy = (orig_point.y, orig_point.x)
target_xy = (target_point.y, target_point.x)

In [0]:
# key: node label ,value: location
idx = index.Index()

xx = nodes['x'].astype(float)
yy = nodes['y'].astype(float)
node_id = nodes['osmid']
node_type = nodes['highway']

# nan, 'crossing', 'turning_circle', 'traffic_signals',
# 'motorway_junction', 'turning_loop', 'elevator', 'bus_stop',
# 'mini_roundabout', 'stop', 'speed_camera'

for node, x, y, kind in zip(node_id, xx, yy, node_type):
    if kind != 'nan' and kind != 'stop' and kind != 'speed_camera':
        idx.insert(int(node), (x,y,x,y))

In [0]:
def find_nearest_nodes(source, dist, idx):
    # Create a bounding box around source of min distance in all directions
    (north, south, east, west) = ox.bbox_from_point(point=source, distance=dist)
    
    candidate_nodes = list(idx.intersection((west, south, east, north)))    
    return candidate_nodes

In [0]:
# Max distance for walking is 700m (10 min * 1.2 m/s)
# Find all nodes within 720 m from candidate sources and candidate destinations
candidate_dest = find_nearest_nodes(target_xy, 700, idx)
candidate_source = find_nearest_nodes(orig_xy, 700, idx)

In [0]:
def path_values(G, sources, destinations, nodes):
    distances = dict()
    var_eud_con = list()
    var_dijk_con = list()
    
    for source in sources:
        x1 = nodes[nodes['osmid']==source]['x'].values[0]
        y1 = nodes[nodes['osmid']==source]['y'].values[0]
        
        for dest in destinations:
            x2 = nodes[nodes['osmid']==dest]['x'].values[0]
            y2 = nodes[nodes['osmid']==dest]['y'].values[0]
            
            # euclidean distance in metres
            ed = 1000 * ((x1-x2)**2 + (y1-y2)**2)**0.5
            
            # dijsktra distance in metres
            d = nx.shortest_path_length(G, source, dest, weight='length', method='dijkstra')
            
            # congested distances in metres
            G.add_node(source, demand = -1)
            G.add_node(dest, demand = 1)

            c_time, _ = nx.capacity_scaling(G, weight='BPR', capacity='capacity')
            cd = c_time * 5.28

            G.add_node(source, demand = 0)
            G.add_node(dest, demand = 0)
            distances[(source,dest)] = {'euclidean': ed, 'dijsktra': d, 'real': cd}

            mean_con_eud = (ed + cd)/2
            v_con_eud = (ed**2 + cd**2)/2 - mean_con_eud**2
            var_eud_con.append(v_con_eud)

            mean_con_d = (d + cd)/2
            v_con_d = (d**2 + cd**2)/2 - mean_con_d**2
            var_dijk_con.append(v_con_d)

            print(source, dest)


    return distances, var_eud_con, var_dijk_con

In [16]:
import statistics 

distances, var_eud_con, var_dijk_con = path_values(G, candidate_source, candidate_dest, nodes)
distances

1318459092 7013732733
1318459092 5175554784
1318459092 5175554795
1318459092 5175554782
1318459092 4574348137
1318459092 5175554790
1318459092 7013732730
1318459092 992571865
1318459092 4663363888
1318459092 5266964778
1318459092 992571805
1318459092 992571815
1318459092 5175554811
1318459092 7011940540
1318459092 5175554838
1318459092 7011940535
1318459092 5266964774
1318459092 5266964781
1318459092 4663372158
1318459092 4663372159
1318459092 4663372160
1318459092 7013732724
1318459092 5175554765
1318459092 7013732721
1318459092 5175554840
1318459092 5175554839
1318459092 5175554810
1318459092 5175554842
1318459092 5175554783
1318459092 4574348136
1318459092 5175554809
1318459092 7013732727
1318459092 4605217482
1318459092 7013732725
1318459092 5175554796
1318459092 5175554803
1318459092 3182619362
1318459092 6281637319
1318459092 4661984546
1318459092 1820026269
1318459092 1820026248
1318459092 6281637317
1318459092 1820026231
1318459092 4660426171
1318459092 6281637324
1318459092 62

{(1318459092, 7013732733): {'dijsktra': 15232.428999999996,
  'euclidean': 110.96364209975148,
  'real': 16598.730740002935},
 (1318459092, 5175554784): {'dijsktra': 15485.557999999999,
  'euclidean': 111.2016204310446,
  'real': 16741.61420230694},
 (1318459092, 5175554795): {'dijsktra': 15528.533999999996,
  'euclidean': 111.20963611171408,
  'real': 16735.39686291494},
 (1318459092, 5175554782): {'dijsktra': 15519.673999999999,
  'euclidean': 111.507536504488,
  'real': 16733.58998144294},
 (1318459092, 4574348137): {'dijsktra': 15514.236999999996,
  'euclidean': 111.50155878237382,
  'real': 16729.146443266938},
 (1318459092, 5175554790): {'dijsktra': 15478.231999999996,
  'euclidean': 111.47894494572272,
  'real': 16713.405633346938},
 (1318459092, 7013732730): {'dijsktra': 15298.430999999997,
  'euclidean': 111.41581900117154,
  'real': 16634.799512962938},
 (1318459092, 992571865): {'dijsktra': 15281.424999999996,
  'euclidean': 111.40405338703543,
  'real': 16616.580962722936},

In [17]:
print(var_eud_con)
print(statistics.mean(var_eud_con))

[67961615.96867469, 69142655.66085403, 69090900.23795187, 69073406.20158277, 69036530.37670901, 68905992.44176212, 68255552.17447037, 68105216.20211874, 68190112.2186694, 69273445.25306751, 68164734.8197232, 68132667.03944786, 69166045.32338873, 69977015.50130829, 69130111.61267017, 68220028.57708779, 69713694.99435405, 74250475.65078533, 67497518.57632053, 67308627.26025933, 67785712.41674435, 66873934.97681114, 66989086.36773476, 67095533.159863785, 67114437.5295912, 67937087.20663682, 67897527.62716267, 67151416.8454289, 67973778.30195053, 68013402.4602607, 68130172.07841894, 68809148.03047912, 67029464.371712685, 67124327.6403922, 68103981.93921918, 70162547.27470736, 67075697.98541856, 65691059.52063398, 65729288.31761441, 68133550.91562715, 65256715.20286435, 67401855.55593756, 65090662.467488624, 64727706.40847819, 65545664.486549914, 65840992.50179428, 64787629.71102354, 64764895.82544872, 65343165.270548895, 65160056.3943752, 65230483.31179013, 65258008.036608055, 66384836.973

In [18]:
print(var_dijk_con)
print(statistics.mean(var_dijk_con))

[466695.11118376255, 394419.2958384752, 364129.4924708307, 368398.00250062346, 369001.23883482814, 381413.4761288762, 446470.200609684, 445660.3611986637, 433618.8051021993, 349758.36752587557, 437713.7627026439, 441697.14622858167, 359638.9761917591, 288583.8862093687, 362976.3199611306, 429207.93584665656, 386328.2018225789, 587737.3311869502, 634751.8532439768, 664116.1206436753, 498404.4677310884, 642621.6463347077, 622594.4996046424, 623705.3519678414, 601505.3852389455, 523777.2240663767, 528266.1402340233, 599866.4909199178, 519552.0563042164, 515109.2837894261, 502134.69201540947, 430163.67138427496, 615651.01838395, 599501.2250868976, 504705.5752016008, 272927.6317014992, 609053.262503624, 1327076.049676627, 1382441.292637229, 508352.2582651973, 1046159.4797243178, 852460.7969174981, 1075260.567863524, 1309999.0724339485, 1087326.7356731892, 1353721.6174352467, 1295035.8512955606, 1301737.6710900366, 995653.4933854938, 1051813.7551115155, 1028264.2124684453, 1022948.144556433,

In [0]:
# Variance of dijkstra - congested is lesser